In [1]:
from music21 import converter, instrument, note, chord, stream, pitch
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM  
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
def parse_midi_file(midi_sample):

    notes_piano = []
    metadata_piano = {
        "note_count": 0,
        "chord_count": 0,
        "rest": 0,
        "else_count": 0
    }
    else_arr = []

#     midi_sample = converter.parse(file)
    instruments = instrument.partitionByInstrument(midi_sample)

    for part in instruments.parts:
        # print(str(part))                          #show Piano Templat

        if 'Piano' in str(part):

            notes_to_parse = part.recurse()
            print(len(notes_to_parse))            #show Piano Template len (7)
            print('m')

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    if str(element.duration.type) == 'complex':
                        notes_piano.append('n_'+str(element.pitch)+'_'+str(element.duration.components[0].type))
                    else:
                        notes_piano.append('n_'+str(element.pitch)+'_'+str(element.duration.type))
                    metadata_piano["note_count"] += 1

                elif isinstance(element, chord.Chord):
                    notes_piano.append('.'.join(str(n) for n in element.normalOrder))
                    metadata_piano["chord_count"] += 1

                elif isinstance(element, note.Rest):
                    if str(element.duration.type) == 'complex':
                        notes_piano.append('r_' + str(element.duration.components[0].type))
                    else:
                        notes_piano.append('r_'+str(element.duration.type))
                    metadata_piano["rest"] += 1
                else:
                    metadata_piano["else_count"] += 1
                    else_arr.append(element)

    print('p')
    return notes_piano, metadata_piano, else_arr

In [3]:
def mapping(notes):
    sequence_length = 10        #100

    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note_var, number) for number, note_var in enumerate(pitchnames))

    nn_input = []
    nn_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]

        nn_input.append([note_to_int[char] for char in sequence_in])
        nn_output.append(note_to_int[sequence_out])
    n_patterns = len(nn_input)

    # # reshape the input into a format compatible with LSTM layers
    nn_input = np.reshape(nn_input, (n_patterns, sequence_length, 1))
    # # normalize input
    nn_input = nn_input / float(n_patterns)

    nn_output = np_utils.to_categorical(nn_output)

    return nn_input, nn_output, note_to_int, pitchnames


In [4]:
def save_midi_file(output, mapping_keys):

    # n = note.Note(str(element.pitches).split(' ')[-1][:-3], type=element.duration.type)
    # s.append(n)
    # s.show('text')print('text')

    unmapped_from_int = []
    converted = []

   # unmapping notes, chores and rests from output integers
    for element in output:
        index = np.where(element)[0][0]
        for key in mapping_keys:
            if int(mapping_keys[key]) == index:
                unmapped_from_int.append(key)

    # creating note, chores and rest objects
    for element in unmapped_from_int:
        if '.' in element:                          #chors
            converted.append(chord.Chord(element))
        elif 'n_' in element:                       #note
            element = element[2:]
            note_ = note.Note(element.split('_')[0])        #creating note
            note_.duration.type = element.split('_')[1]     #adding duration type
            converted.append(note_)                         #appending array

        elif 'r_' in element:                       #rest
            element = element[2:]
            # rest = note.Rest()
            # rest.duration.quarterLength = 2.0
            # converted.append(rest)
            converted.append(note.Rest(type=element))

    print(converted)

    try:
        midi_stream = stream.Stream()
        midi_stream.append(converted)
        midi_stream.write('midi', fp='midi_samples\\outputs\\test_output.mid')
        print('created new MIDI file')
    except:
        print('save_midi_file ERROR')

In [5]:
midi_file = converter.parse('midi_samples\\cosmo-27notes.mid')
midi_file.show('midi')

In [6]:

notes_and_chords, metadata_p, else_array = parse_midi_file(midi_file)

print('main')
print(notes_and_chords)
print(metadata_p)
print('koniec mojho')

lstm_input, lstm_output, notes_to_int, pitch_names = mapping(notes_and_chords)

save_midi_file(lstm_output, notes_to_int)

47
m
p
main
['n_E5_eighth', 'r_16th', 'n_E5_eighth', 'r_16th', 'n_E5_quarter', 'r_16th', 'n_D5_eighth', 'n_E5_eighth', 'n_G5_half', 'r_quarter', 'n_G5_eighth', 'n_A5_eighth', 'n_G5_eighth', 'n_B5_eighth', 'r_16th', 'n_B5_eighth', 'r_16th', 'n_B5_eighth', 'n_A5_eighth', 'n_G5_eighth', 'n_A5_eighth', 'n_A5_eighth', 'n_B5_half', 'r_16th', 'n_A5_quarter', 'r_16th', 'n_G5_eighth', 'n_A5_eighth', 'n_B5_quarter', 'r_16th', 'n_G5_eighth', 'n_E5_eighth', 'n_E5_quarter', 'r_16th', 'n_D5_eighth', 'r_16th', 'n_G5_eighth', 'n_E5_eighth', 'n_D5_eighth', 'n_E5_half', 'n_E5_half', 'r_quarter']
{'note_count': 30, 'chord_count': 0, 'rest': 12, 'else_count': 5}
koniec mojho
[<music21.note.Note G>, <music21.note.Note A>, <music21.note.Note G>, <music21.note.Note B>, <music21.note.Rest 16th>, <music21.note.Note B>, <music21.note.Rest 16th>, <music21.note.Note B>, <music21.note.Note A>, <music21.note.Note G>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note B>, <music21.note.Rest 16th>, <mus

In [7]:
new_midi_file = converter.parse('midi_samples\\outputs\\test_output.mid')
new_midi_file.show('midi')

In [32]:
c3 = chord.Chord(['E5', 'A5', 'D3', 'A3', 'B-3', 'E4'])
c2 = c3.normalOrder
print(c3)
print(c2)

notes = []
for current_note in c2:

    cn=int(current_note)
    new_note = note.Note(cn)
    new_note.storedInstrument = instrument.Piano()
    notes.append(new_note)

new_chord = chord.Chord(notes)
new_chord.offset = 0
print(new_chord)

# c3.show('midi')
new_chord.show('midi')

<music21.chord.Chord E5 A5 D3 A3 B-3 E4>
[9, 10, 2, 4]
<music21.chord.Chord A B- D E>
